The two-dimensional turbulence simulation over a spatially periodic domain $ U = [0, 2\pi] \times [0, 2\pi] $ is obtained from a pseudo-spectral code applied to the 2-D, incompressible Navier–Stokes equation (see [1]) with viscosity ($ \mu = 2 \times 10^{-5} $). The spatial coordinates are resolved using $ 1024 \times 1024 $ Fourier modes and a $ 2/3 $ dealiasing. The velocity field was derived over the time-interval $ [0, 50] $ and contains 251 equally spaced velocity field snapshots. 

The following table highlights the variables stored in the mat-file '2DTurbulence.mat'.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| x | array (Nx,) | x-coordinate $ [0, 2\pi] $ with Nx = 1024 |
| y | array (Ny,) | y-coordinate $ [0, 2\pi] $ with Ny = 1024 |
| t | array (N,) | time $ [0, 250] $ with N = 251 |
| U | array (Ny, Nx, N) | x-component of velocity |
| V | array (Ny, Nx, N) | y-component of velocity |
| omega | array (Ny, Nx, N) | vorticity |

[1] Farazmand, M. M., Kevlahan, N. R., & Protas, B. (2011). Controlling the dual cascade of two-dimensional turbulence. $ \textit{Journal of fluid mechanics, 668, 202-222.} $

In [1]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

Wall time: 0 ns


In [2]:
# Import scipy for loading/saving mat-files
import scipy.io

# Import os
import os

# Import numpy
import numpy as np

In [3]:
%%time
import scipy.io as sio

# get current directory
current_directory = os.getcwd()

# load mat-file
mat_data = scipy.io.loadmat(current_directory + r'/Turbulence.mat')

# load variables
U = mat_data['U'][:,:,:26]
V = mat_data['V'][:,:,:26]
omega = mat_data['omega']
x = mat_data['x']
y = mat_data['y']
time_data = mat_data['t'][:,:26]

Wall time: 14.9 s


# Compute trajectories

We precompute the trajectories in the periodic two-dimensional turbulence model. The trajectories are computed over the spatial domain:

\begin{equation}
\lbrace x, y \in [0, 2\pi] \times [0, 2\pi] \rbrace
\end{equation} over the time-interval $ t \in [0, 25] $

## Computational parameters and data

Here we define the computational parameters and the data.

In [4]:
import numpy as np

# number of cores to be used for parallel computing
Ncores = 8

# time resolution of data
dt_data = time_data[0,1]-time_data[0,0]

# periodic boundary conditions
periodic_x = False
periodic_y = False
periodic = [periodic_x, periodic_y]

# unsteady velocity field
bool_unsteady = True

# defined domain
# This array denotes where the velocity field is defined (=1) and where it is not defined (=0)
defined_domain = np.isfinite(U[:,:,0]).astype(int)

## meshgrid
X, Y = np.meshgrid(x, y)

## resolution of meshgrid
dx_data = X[0,1]-X[0,0]
dy_data = Y[1,0]-Y[0,0]

delta = [dx_data, dy_data]

## Spatio-temporal domain

Here we define the spatio-temporal domain over which to consider the dynamical system.

In [5]:
%%time
# Initial time (in days)
t0 = 0

# Final time (in days)
tN = 25

# time step-size (in days)
dt = .2

time = np.arange(t0, tN+dt, dt)

# length of time interval (in days)
lenT = abs(tN-t0)

# longitudinal and latitudinal boundaries (in degrees)
xmin = 0
xmax = 2*np.pi
ymin = 0
ymax = 2*np.pi

# make sure that the chosen domain is part of the data domain
assert (xmax <= np.max(X) and xmin >= np.min(X) and ymin >= np.min(Y) and ymax <= np.max(Y) and t0 >= np.min(time_data) and tN <= np.max(time_data)),"The domains you are chooising are outside the domain of the data!!!!! --> redefine spatial/temporal domain"

# Resolution of meshgrid (in degrees)
Nx = 1024
Ny = 1024

x_domain = np.linspace(xmin, xmax, Nx, endpoint = True)
y_domain = np.linspace(ymin, ymax, Ny, endpoint = True)

X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

Wall time: 56 ms


## Interpolate velocity

In order to evaluate the velocity field at arbitrary locations and times, we interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear".

In [6]:
%%time
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# set nan values to zero so that we can apply interpolant. Interpolant does not work if the array contains nan values
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, U, V, time_data, method = "cubic")

Wall time: 17.7 s


## Compute trajectories over meshgrid

In [ ]:
%%time
# Import package for computing trajectories/velocity along trajectories
from ipynb.fs.defs.integration_dFdt import integration_dFdt

# Import package which checks particle location
from ipynb.fs.defs.check_location import check_location

# Import package for progress bar
from tqdm.notebook import tqdm

# Import package for parallel computing
from joblib import Parallel, delayed

import matplotlib.pyplot as plt

def parallel_trajectories(i):
    
    Trajectories_parallel = np.zeros((X_domain.shape[1], 2, len(time)))
    
    for j in range(X_domain.shape[1]):
        
        # set initial condition
        x = np.array([X_domain[i, j], Y_domain[i, j]])

        # compute velocity along trajectory
        Fmap = integration_dFdt(time, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta)[0]
        
        fig = plt.figure()
        ax = plt.axes()
        ax.plot(Fmap[0,:], Fmap[1,:])
        plt.show()
    
        Trajectories_parallel[j,:,:] = Fmap
        
    return Trajectories_parallel[j,:,:]

Ncores
Trajectories = np.array(Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel_trajectories)(i) for i in tqdm(range(X_domain.shape[0]))))

  0%|          | 0/1024 [00:00<?, ?it/s]

In [ ]:
#scipy.io.savemat(file_processed_data, {'': u, 'V': v, 'omega': v, 'x': x, 'y': y, 't': t})